In [2]:
import boto3
import pandas as pd
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
from sagemaker import get_execution_role
from botocore.client import Config

### Leer los datos del bucket del proyecto en la zona Raw

In [3]:
if sys.version_info[0] < 3: 
    from StringIO import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

client = boto3.client('s3')
bucket_name = 'prr-integrador-grupo3'
object_key1 = 'Zona-raw/Precios/Precios por modelo_prov.csv'
object_key2 = 'Zona-raw/RUNT/runt_Diario.txt'
object_key3 = 'Zona-raw/Segmentos/Segmentos_por_modelo.csv'


In [4]:
csv_obj1 = client.get_object(Bucket=bucket_name, Key=object_key1)
csv_obj2 = client.get_object(Bucket=bucket_name, Key=object_key2)
csv_obj3 = client.get_object(Bucket=bucket_name, Key=object_key3)

In [5]:
body1 = csv_obj1['Body']
body2 = csv_obj2['Body']
body3 = csv_obj3['Body']

csv_string1 = body1.read().decode('utf-8')
csv_string2 = body2.read().decode('utf-8')
csv_string3 = body3.read().decode('utf-8')

precios = pd.read_csv(StringIO(csv_string1))
data = pd.read_csv(StringIO(csv_string2), sep = '\t')
segmentos = pd.read_csv(StringIO(csv_string3))

In [7]:
data.head(5)

,Categoría,Combustible,Compañía,Día natural,Marca,Modelo,Cant. Total
0,MOTORCYCLES,GASOLINA,YAMAHA,16/03/2021,YAMAHA,XTZ125,1
1,MOTORCYCLES,GASOLINA,YAMAHA,16/03/2021,YAMAHA,XTZ125,1
2,MOTORCYCLES,GASOLINA,YAMAHA,16/03/2021,YAMAHA,XTZ125,1
3,MOTORCYCLES,GASOLINA,YAMAHA,16/03/2021,YAMAHA,XTZ125,1
4,MOTORCYCLES,GASOLINA,YAMAHA,16/03/2021,YAMAHA,XTZ125,1


In [8]:
precios['FECHA'] =  pd.to_datetime(precios['FECHA'], infer_datetime_format=True)
data['Día natural']  = pd.to_datetime(data['Día natural'], infer_datetime_format=True)

In [9]:
#Compañias
data.Compañía.unique()

array(['YAMAHA', 'OTRAS', 'UMA', 'AUTECO', 'HONDA', 'HERO', 'AKT',
       'SUZUKI', 'MOBILITY', 'Sin asignar'], dtype=object)

In [10]:
#Marcas
data.Marca.unique()

array(['YAMAHA', 'BAJAJ', 'HONDA', 'AKT', 'SUZUKI', 'VICTORY', 'TVS',
       'HERO', 'KYMCO', 'KTM', 'BENELLI', 'ROYAL ENFIELD', 'KAWASAKI',
       'SYM', 'PIAGGIO', 'HM HONDA', 'SIN DEFINIR', 'HUSQVARNA', 'HAOJUE',
       'AYCO', 'VESPA', 'TONGKO', 'ZAGA', 'BADER', 'AVA MOTORS', 'WEST',
       'LINTEX', 'BK MOTO', 'SUPER SOCO', 'GVMCO', 'TVS-TYRES', 'APRILIA',
       'BOSS ATV', 'MOTO ABC', 'POLARIS', 'LARETTO', 'ZNEN', 'JIALING',
       'UM', 'YGM', 'KAZUKI', 'UNITED MOTORS', 'LML', 'MV AGUSTA',
       'CAN-AM', 'SENKE', 'BETA', 'TX MOTORS', 'BOSS', 'TRIUMPH', 'ASA',
       'LINHAI', 'ELEKTROMOTORES', 'MOTO GUZZI', 'N.S.U', 'LMX',
       'REVOLUTION 250', 'SCOMADI', 'STARKER', 'ARCTIC CAT', 'HUSABERG',
       'LIFAN', 'HUASHA', 'WEST PANTERA', 'SUNRA', 'JINLING',
       'HARLEY DAVIDSON', 'BRP/CAN AM', 'MACBOR', 'CFMOTO', 'DUCATI',
       'SIGMA', 'KEEWAY', 'BMW', 'NIU', 'VOLTA', 'ENERGY MOTION',
       'BIOLOGICA', 'ZERO', 'E-RACE', 'WOLF', 'ELECTRIC E-BIKE GX',
       'ELECTRIKA', 

In [11]:
data.groupby('Compañía').count()

,Categoría,Combustible,Día natural,Marca,Modelo,Cant. Total
Compañía,,,,,,
AKT,360900,360900,360900,360900,360900,360900
AUTECO,302695,302695,302695,302695,302695,302695
HERO,109481,109481,109481,109481,109481,109481
HONDA,291653,291653,291653,291653,291653,291653
MOBILITY,346095,346095,346095,346095,346095,346095
OTRAS,32513,32513,32513,32513,32513,32513
SUZUKI,260329,260329,260329,260329,260329,260329
Sin asignar,29,29,29,29,29,29
UMA,277154,277154,277154,277154,277154,277154


### Eliminación de Outliers

In [12]:
#Eliminamos de dataframe algunos registros erroneos
new_data = data[~(data.Modelo.isin(['#', 'NO TIENE', 'SIN REFERENCIA', 'SIN DEFINIR'])) & ~(data.Compañía.isin(['Sin asignar', 'OTRAS'])) & ~(data.Marca.isin(['Sin asignar', 'SIN DEFINIR']))].reset_index(drop=True)

In [13]:
new_data.head(5)

,Categoría,Combustible,Compañía,Día natural,Marca,Modelo,Cant. Total
0,MOTORCYCLES,GASOLINA,YAMAHA,2021-03-16,YAMAHA,XTZ125,1
1,MOTORCYCLES,GASOLINA,YAMAHA,2021-03-16,YAMAHA,XTZ125,1
2,MOTORCYCLES,GASOLINA,YAMAHA,2021-03-16,YAMAHA,XTZ125,1
3,MOTORCYCLES,GASOLINA,YAMAHA,2021-03-16,YAMAHA,XTZ125,1
4,MOTORCYCLES,GASOLINA,YAMAHA,2021-03-16,YAMAHA,XTZ125,1


### Filtro por subsegmentos y construcción del dataframe para evaluar modelo

In [14]:
df_segmento = segmentos[(segmentos.subsegmento == 'COMMUTER ENTRY')].reset_index(drop=True)
df_segmento_list = df_segmento.Modelo.unique()

In [15]:
df_segmento_list

array(['AK 125 SL', 'AK125 NE', 'AK125 NKDR', 'AK125CR4', 'AK125CR4 EIII',
       'AK125NKD EIII', 'AK125RL', 'AX 100', 'AX 4', 'AX4 EVOLUTION',
       'AY 125 2EV', 'AY 125-7', 'AY100', 'BOMBER 125',
       'BOXER BM100 CLASSIC ES', 'BOXER CARGO CT 100',
       'BOXER CARGO CT 100 AHO', 'BOXER CARGO CT 100 ES', 'BOXER CT 100',
       'BOXER CT 100 AHO', 'BOXER CT 100 ED. ANIVER', 'BOXER CT 100 ES',
       'BOXER CT 100 ES TITANNIUM', 'BOXER CT 100 KS',
       'BOXER CT 100 TITANNIUM', 'BOXER CT 125 ES', 'BOXER S', 'BS125',
       'CB 110', 'CB 110 DLX', 'CITY100', 'COMBAT 125', 'CT100 ES ALLOY',
       'CT100 ES SPOKE', 'CT100 KS SPOKE', 'CT125', 'CT125 ALLOY',
       'DISCOVER 100 SPORT', 'DISCOVER 100S', 'DISCOVER 110',
       'DISCOVER 125 UG', 'DISCOVER 125 UG CBS', 'DREAM NEO',
       'DREAM NEO ACTIVA', 'ECO', 'ECO 100', 'ECO DELUXE',
       'ECO DELUXE CW', 'ECO DELUXE ES', 'ECO DELUXE I3S',
       'ECO DELUXE SP', 'GE110', 'GLAMOUR', 'GN 125', 'GN125H', 'GV-100',
       'HAYAT

In [16]:
data_segmento = new_data[(new_data.Modelo.isin(df_segmento_list))].reset_index(drop=True)
precios_segmento = precios[(precios.MODELO.isin(df_segmento_list))].reset_index(drop=True)

In [17]:
#Filtramo los datos a partir del 2021 (Post-pandemia)
data_segmento = data_segmento[(data_segmento['Día natural'].dt.year > 2020)].reset_index(drop=True)
#Ordenamos los datos cronologicamente
data_segmento = data_segmento.sort_values('Día natural').reset_index(drop=True)
#Filtramo los datos a partir del 2021 (Post-pandemia)
precios_segmento = precios_segmento[(precios_segmento['FECHA'].dt.year > 2020)].reset_index(drop=True)
#Ordenamos los datos cronologicamente
precios_segmento = precios_segmento.sort_values('FECHA').reset_index(drop=True)

In [18]:
Modelos_entry = precios_segmento.MODELO.unique()
data_segmento = data_segmento[(data_segmento.Modelo.isin(Modelos_entry))].reset_index(drop=True)

In [19]:
data_compañia = data_segmento.groupby(['Compañía', 'Marca', 'Modelo']).agg({'Cant. Total': 'count'})

In [21]:
for i in range(len(data_compañia)):
    print('Referencia: {}, Cant Total = {}'.format(data_compañia.index[i], data_compañia['Cant. Total'][data_compañia.index[i]]))

Referencia: ('AKT', 'AKT', 'AK125CR4'), Cant Total = 5
Referencia: ('AKT', 'AKT', 'AK125CR4 EIII'), Cant Total = 31246
Referencia: ('AKT', 'AKT', 'AK125NKD EIII'), Cant Total = 84397
Referencia: ('AKT', 'TVS', 'SPORT 100 ELS'), Cant Total = 13
Referencia: ('AKT', 'TVS', 'SPORT ELS SPOKE'), Cant Total = 8
Referencia: ('AKT', 'TVS', 'SPORT KLS SPOKE'), Cant Total = 6
Referencia: ('AKT', 'TVS', 'STRYKER 125 NG'), Cant Total = 13
Referencia: ('AUTECO', 'BAJAJ', 'BOXER CT 100 ES'), Cant Total = 3150
Referencia: ('AUTECO', 'BAJAJ', 'BOXER CT 100 ES TITANNIUM'), Cant Total = 932
Referencia: ('AUTECO', 'BAJAJ', 'BOXER CT 100 KS'), Cant Total = 449
Referencia: ('AUTECO', 'BAJAJ', 'BOXER S'), Cant Total = 1026
Referencia: ('AUTECO', 'TVS', 'MAX125 SW'), Cant Total = 2205
Referencia: ('AUTECO', 'TVS', 'SPORT 100 ELS'), Cant Total = 26197
Referencia: ('AUTECO', 'TVS', 'SPORT 100 KLS'), Cant Total = 7026
Referencia: ('AUTECO', 'TVS', 'SPORT ELS SPOKE'), Cant Total = 5826
Referencia: ('AUTECO', 'TVS

In [22]:
week = []
for i in range(len(data_segmento)):
    week.append('{}-{}'.format(data_segmento['Día natural'][i].isocalendar()[0],data_segmento['Día natural'][i].isocalendar()[1]))

In [23]:
data_segmento['week'] = week

In [24]:
data_segmento.head(5)

,Categoría,Combustible,Compañía,Día natural,Marca,Modelo,Cant. Total,week
0,MOTORCYCLES,GASOLINA,AUTECO,2021-01-04,BAJAJ,BOXER S,1,2021-1
1,MOTORCYCLES,GASOLINA,AKT,2021-01-04,AKT,AK125NKD EIII,1,2021-1
2,MOTORCYCLES,GASOLINA,AUTECO,2021-01-04,BAJAJ,BOXER CT 100 ES,1,2021-1
3,MOTORCYCLES,GASOLINA,AUTECO,2021-01-04,BAJAJ,BOXER CT 100 ES,1,2021-1
4,MOTORCYCLES,GASOLINA,AKT,2021-01-04,AKT,AK125NKD EIII,1,2021-1


In [25]:
precios_full = ['AK125CR4 EIII', 'ECO DELUXE CW', 'GN 125','BOXER S','YC110D (YC-Z)', 'SPORT 100 ELS' ] #AKT, Honda, Susuki, UMA, YAMAHA, Y=AUTECO

In [26]:
data_segmento_1 = data_segmento[(data_segmento.Modelo.isin(precios_full))].reset_index(drop=True)
precios_segmento_1 = precios_segmento[(precios_segmento.MODELO.isin(precios_full))].reset_index(drop=True)

In [28]:
precios_segmento_1.head(5)

,MODELO,PRECIO,FECHA,VARIACION_PRECIO
0,YC110D (YC-Z),4750000,2021-01-01,0.0
1,SPORT 100 ELS,3890000,2021-01-01,0.0
2,AK125CR4 EIII,4690000,2021-01-01,0.0
3,GN 125,5188000,2021-01-01,0.0
4,AK125CR4 EIII,4690000,2021-01-02,0.0


In [ ]:
price = []
for i in range(len(data_segmento_1)):
    list_tmp = []
    date_1 = data_segmento_1['Día natural'][i]
    df_tmp = precios_segmento_1[(precios_segmento_1.MODELO == data_segmento_1.Modelo[i])].reset_index(drop=True)
    for j in range(len(df_tmp)):
        date_2 = df_tmp['FECHA'][j]
        list_tmp.append(np.abs(date_1 - date_2))
    
    price.append(df_tmp.PRECIO[list_tmp.index(min(list_tmp))])

In [ ]:
data_segmento_1['precio'] = price

In [ ]:
weeks = data_segmento_1.week.unique()

In [ ]:
data_segmento_1[(data_segmento_1.Modelo == 'SPORT 100 ELS')].reset_index(drop=True)

In [ ]:
results = []
for week in weeks:
    Moto_1 = data_segmento_1[(data_segmento_1.Modelo == 'AK125CR4 EIII') & (data_segmento_1.week == week)].reset_index(drop=True)
    Moto_2 = data_segmento_1[(data_segmento_1.Modelo == 'ECO DELUXE CW') & (data_segmento_1.week == week)].reset_index(drop=True)
    Moto_3 = data_segmento_1[(data_segmento_1.Modelo == 'GN 125') & (data_segmento_1.week == week)].reset_index(drop=True)
    Moto_4 = data_segmento_1[(data_segmento_1.Modelo == 'BOXER S') & (data_segmento_1.week == week)].reset_index(drop=True)
    Moto_5 = data_segmento_1[(data_segmento_1.Modelo == 'YC110D (YC-Z)') & (data_segmento_1.week == week)].reset_index(drop=True)
    target = data_segmento_1[(data_segmento_1.Modelo == 'SPORT 100 ELS') & (data_segmento_1.week == week)].reset_index(drop=True)
    
    result = {'date':week, 'AK125CR4 EIII(price)': np.mean(Moto_1['precio']),'ECO DELUXE CW(price)': np.mean(Moto_2['precio']), 
              'GN 125(price)': np.mean(Moto_3['precio']), 'BOXER S(price)': np.mean(Moto_4['precio']), 'YC110D (YC-Z)(price)': np.mean(Moto_5['precio']),
              'AK125CR4 EIII(unidades)': sum(Moto_1['Cant. Total']),'ECO DELUXE CW(unidades)': sum(Moto_2['Cant. Total']), 
              'GN 125(unidades)': sum(Moto_3['Cant. Total']), 'BOXER S(unidades)': sum(Moto_4['Cant. Total']), 'YC110D (YC-Z)(unidades)': sum(Moto_5['Cant. Total']),
              'SPORT 100 ELS(y)': sum(target['Cant. Total']), 'SPORT 100 ELS(price)': np.mean(target['precio'])}
    results.append(result)

In [ ]:
df_COMMUTER_ENTRY = pd.DataFrame(results)

In [ ]:
df_COMMUTER_ENTRY.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax.plot(df_COMMUTER_ENTRY.date, df_COMMUTER_ENTRY['AK125CR4 EIII(price)'], label = 'AK125CR4 EIII(unidades)')
ax.plot(df_COMMUTER_ENTRY.date, df_COMMUTER_ENTRY['ECO DELUXE CW(price)'], label = 'ECO DELUXE CW(unidades)')
ax.plot(df_COMMUTER_ENTRY.date, df_COMMUTER_ENTRY['GN 125(price)'], label = 'GN 125(unidades)')
ax.plot(df_COMMUTER_ENTRY.date, df_COMMUTER_ENTRY['BOXER S(price)'], label = 'BOXER S(unidades)')
ax.plot(df_COMMUTER_ENTRY.date, df_COMMUTER_ENTRY['YC110D (YC-Z)(price)'], label = 'YC110D (YC-Z)(unidades)')
ax.plot(df_COMMUTER_ENTRY.date, df_COMMUTER_ENTRY['SPORT 100 ELS(price)'], label = 'SPORT 100 ELS(y)')
plt.legend()
plt.show()

### Almacenar el dataframe procesado en la Zona Trusted del bucket

In [ ]:
#### guardar info en el bucket
BUCKET_NAME ='prr-integrador-grupo3'
s3= boto3.resource('s3')
data= df_COMMUTER_ENTRY
s3.Bucket(BUCKET_NAME).put_object(Key='Zona-trusted/df_COMMUTER_ENTRY.csv',Body=data)